In [1]:
# SMARTKNOWER MACHINE LEARNING MAY BATCH MAJOR PROJECT ON SENTIMENT ANALYSIS
# Submitted By Sagar Tanwar (sagtan1297@gmail.com)

In [2]:
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Major Project/Twitter.csv')
df.head()

,Unnamed: 0,clean_text,category
0,0,when modi promised “minimum government maximum...,-1.0
1,1,talk all the nonsense and continue all the dra...,0.0
2,2,what did just say vote for modi welcome bjp t...,1.0
3,3,asking his supporters prefix chowkidar their n...,1.0
4,4,answer who among these the most powerful world...,1.0


In [5]:
df['category'].value_counts()

 1.0    8374
 0.0    6807
-1.0    4819
Name: category, dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  20000 non-null  int64  
 1   clean_text  19999 non-null  object 
 2   category    20000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 468.9+ KB


In [7]:
df.isna().sum()

Unnamed: 0    0
clean_text    1
category      0
dtype: int64

In [8]:
df.dropna(axis = 0, inplace = True)
df.isna().sum()

Unnamed: 0    0
clean_text    0
category      0
dtype: int64

In [9]:
df = df.iloc[:50000, ]

In [10]:
x = df['clean_text'].values
y = df['category'].astype(str).values

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=103)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vect = CountVectorizer(stop_words='english')
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [14]:
x_train_vect = vect.fit_transform(x_train.astype("U"))
x_test_vect = vect.transform(x_test)

In [15]:
from sklearn.svm import SVC
model = SVC()
model.fit(x_train_vect,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
y_pred = model.predict(x_test_vect)

In [17]:
from sklearn.metrics import accuracy_score,classification_report

In [18]:
accuracy_score(y_pred,y_test)

0.7333333333333333

In [19]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

        -1.0       0.47      0.81      0.60       852
         0.0       0.87      0.71      0.78      2515
         1.0       0.78      0.73      0.75      2633

    accuracy                           0.73      6000
   macro avg       0.71      0.75      0.71      6000
weighted avg       0.77      0.73      0.74      6000



In [20]:
test = df['clean_text'][658]
test

'urge all south states india people support all respect with heart mind for modi have lagged behind nation after independence due neta propaganda for religion language local strategycastescreedsregions bias kith kind etc'

In [21]:
df['category'][658]

-1.0

In [22]:
test = vect.transform([df['clean_text'][658]])
model.predict(test)

array(['-1.0'], dtype=object)

In [23]:
from sklearn.pipeline import make_pipeline
text_model = make_pipeline(CountVectorizer(),SVC())
 
text_model.fit(x_train,y_train)
 
y_pred = text_model.predict(x_test)
y_pred

array(['0.0', '1.0', '0.0', ..., '1.0', '0.0', '0.0'], dtype=object)

In [24]:
text_model.predict([df['clean_text'][0]])

array(['-1.0'], dtype=object)

In [25]:
text_model

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='rbf', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
        

In [26]:
import joblib
joblib.dump(text_model,'/content/drive/MyDrive/Major Project/Sentiment-Analysis.joblib')

['/content/drive/MyDrive/Major Project/Sentiment-Analysis.joblib']

In [27]:
%%writefile app.py
import streamlit as st
from joblib import load
 
#from google.colab import drive
#drive.mount('/content/drive')
 
model = load('/content/drive/MyDrive/Major Project/Sentiment-Analysis.joblib')
 
st.title('Sentiment Analysis')
text = st.text_input('Enter the message')
 
predict = float(model.predict([text])) 
 
category = 'Normal Category'
 
if predict == 1.0:
   category = 'Positive Category'
elif predict == -1.0:
   category = 'Negative Category'
 
st.write(category)

Overwriting app.py


In [28]:
 from pyngrok import ngrok

In [29]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://d460597c403e.ngrok.io'